# All D11 Calculation Pathways

This notebook analyzes snow D11 (flexural rigidity in the x-direction) calculation methods at the **slab-level** scale.

## Table of Contents

1. [Load Snow Pit Data](#1-load-snow-pit-data)
2. [Find All D11 Calculation Pathways](#2-find-all-d11-calculation-pathways)
3. [Slab-Level Analysis (ECTP)](#3-slab-level-analysis-ectp)
4. [Sankey Diagram: kim_jamieson_table2 → wautier → kochle](#4-sankey-diagram-kim_jamieson_table2--wautier--kochle)

**Target Parameter**: `D11` — slab flexural rigidity in the x-direction (N·m)

D11 is a slab-level property requiring elastic modulus and Poisson's ratio for all layers. Uncertainty reflects propagated input measurement uncertainties only (method regression standard error excluded).

In [ ]:
from pathlib import Path
from typing import Dict, Any
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from snowpyt_mechparams.snowpilot import parse_caaml_directory
from snowpyt_mechparams.data_structures import Pit, Slab
from snowpyt_mechparams.graph import graph
from snowpyt_mechparams.algorithm import find_parameterizations
from snowpyt_mechparams.execution import ExecutionEngine
from snowpyt_mechparams.execution.config import ExecutionConfig

## 1. Load Snow Pit Data

In [ ]:
snow_pits_raw = parse_caaml_directory(str(Path("data")))
pits = [Pit.from_snow_pit(sp) for sp in snow_pits_raw]

print(f"Loaded {len(pits)} snow pits ({sum(len(pit.layers) for pit in pits)} layers)")

## 2. Find All D11 Calculation Pathways

In [ ]:
pathways = find_parameterizations(graph, graph.get_node("D11"))

print(f"Found {len(pathways)} pathways for calculating D11:\n")
for i, pathway in enumerate(pathways, 1):
    print(f"Pathway {i}:")
    print(pathway)
    print()

### Why 32 Pathways, Not More?

D11 requires three layer-level parameters, each with multiple methods:

| Parameter | Methods | Count |
|-----------|---------|-------|
| `density` | `data_flow`, `geldsetzer`, `kim_jamieson_table2`, `kim_jamieson_table5` | 4 |
| `elastic_modulus` | `bergfeld`, `kochle`, `wautier`, `schottner` | 4 |
| `poissons_ratio` | `kochle`, `srivastava` | 2 |

4 × 4 × 2 = **32 unique method combinations**.

The `srivastava` Poisson's ratio method requires `density` as input. The algorithm's backward traversal treats the density sub-path for `elastic_modulus` and for `srivastava` as independent choices, generating up to 80 structural traversals. `find_parameterizations` deduplicates them using `_method_fingerprint`, leaving exactly the 32 unique combinations above.

## 3. Slab-Level Analysis (ECTP)

In [ ]:
# Create ECTP slabs
ectp_slabs = []
for pit in pits:
    for slab in pit.create_slabs(weak_layer_def="ECTP_failure_layer"):
        ectp_slabs.append({'slab': slab, 'n_layers': len(slab.layers)})

print(f"Created {len(ectp_slabs)} ECTP slabs")

In [ ]:
engine = ExecutionEngine(graph)
config = ExecutionConfig(include_method_uncertainty=False)

# For each pathway track per-slab boolean outcomes:
#   density              — all layers computed density
#   e_mod / poissons     — all layers computed E / ν  (independent, parallel)
#   pass_both            — pass E AND pass ν (required for D11)
#   fail_e_only          — pass ν but fail E
#   fail_nu_only         — pass E but fail ν
#   fail_both_ep         — fail both E and ν
#   d11                  — slab D11 computed successfully
pathway_slab_success: Dict[str, int] = {}
pathway_step_counts: Dict[str, Dict[str, int]] = {}
pathway_rel_unc: Dict[str, list] = {}

for info in ectp_slabs:
    slab = info['slab']
    n = info['n_layers']
    results = engine.execute_all(slab, "D11", config=config)
    for pathway_result in results.pathways.values():
        density_method = pathway_result.methods_used.get('density', 'unknown')
        e_mod_method   = pathway_result.methods_used.get('elastic_modulus', 'unknown')
        pr_method      = pathway_result.methods_used.get('poissons_ratio', 'unknown')
        full_pathway   = f"{density_method} \u2192 {e_mod_method} \u2192 {pr_method}"

        if full_pathway not in pathway_slab_success:
            pathway_slab_success[full_pathway] = 0
            pathway_step_counts[full_pathway]  = {
                'density': 0,
                'e_mod': 0, 'poissons': 0,
                'pass_both': 0,
                'fail_e_only': 0, 'fail_nu_only': 0, 'fail_both_ep': 0,
                'd11': 0,
            }
            pathway_rel_unc[full_pathway] = []

        traces = pathway_result.computation_trace
        ok_density  = sum(1 for t in traces if t.parameter == "density"         and t.success and t.output is not None) == n
        ok_e_mod    = sum(1 for t in traces if t.parameter == "elastic_modulus" and t.success and t.output is not None) == n
        ok_poissons = sum(1 for t in traces if t.parameter == "poissons_ratio"  and t.success and t.output is not None) == n

        sc = pathway_step_counts[full_pathway]
        if ok_density:  sc['density']  += 1
        if ok_e_mod:    sc['e_mod']    += 1
        if ok_poissons: sc['poissons'] += 1

        # Parallel step breakdown (only meaningful for slabs that passed density)
        if ok_density:
            if   ok_e_mod and ok_poissons:  sc['pass_both']    += 1
            elif ok_e_mod and not ok_poissons: sc['fail_nu_only']  += 1
            elif ok_poissons and not ok_e_mod: sc['fail_e_only']   += 1
            else:                           sc['fail_both_ep'] += 1

        d11_val = d11_std = None
        for trace in traces:
            if trace.parameter == "D11" and trace.success and trace.output is not None:
                out = trace.output
                if hasattr(out, 'nominal_value'):
                    d11_val, d11_std = out.nominal_value, out.std_dev
                else:
                    try:
                        d11_val, d11_std = float(out), 0.0
                    except (TypeError, ValueError):
                        pass
                break

        if d11_val is not None:
            pathway_slab_success[full_pathway] += 1
            sc['d11'] += 1
            if d11_val != 0 and d11_std is not None:
                pathway_rel_unc[full_pathway].append(d11_std / d11_val)

print(f"Executed {len(pathways)} pathways on {len(ectp_slabs)} slabs")


In [ ]:
total_slabs = len(ectp_slabs)

# Sort by slab count descending
sorted_pathways = sorted(pathway_slab_success.items(), key=lambda x: x[1], reverse=True)

print(f"  {'Full Pathway':<55s} {'Slabs':>30s} {'Avg Rel. Uncertainty':>22s}")
print(f"  {'-'*109}")
for pathway, n_ok in sorted_pathways:
    slab_str = f"{n_ok} / {total_slabs} ({n_ok / total_slabs:.1%})"
    rel_uncs = pathway_rel_unc.get(pathway, [])
    avg_unc = np.mean(rel_uncs) if rel_uncs else float('nan')
    unc_str = f"{avg_unc:.1%}" if not np.isnan(avg_unc) else "N/A"
    print(f"  {pathway:<55s} {slab_str:>30s}    {unc_str:>18s}")

print()
print("  Note: Uncertainty is propagated from input measurement uncertainties only.")
print("  Slab success requires D11 calculation for the entire slab.")


## 4. Sankey Diagram: kim_jamieson_table2 → wautier → kochle

Slabs lost at each successive step of the calculation chain for the pathway
`kim_jamieson_table2 → wautier → kochle`.

In [ ]:
import plotly.graph_objects as go

TARGET          = "kim_jamieson_table2 → wautier → kochle"
METHOD_DENSITY  = "kim_jamieson_table2"
METHOD_EMOD     = "wautier"
METHOD_POISSONS = "kochle"
METHOD_D11      = "weissgraeber_rosendahl"

n_start      = len(ectp_slabs)
sc           = pathway_step_counts[TARGET]

# Sequential step counts (algorithm order: density → E-mod → ν → D11)
n_density    = sc['density']     # slabs where ALL layers got density
n_e_mod      = sc['e_mod']       # slabs where ALL layers got E-mod
n_poissons   = sc['poissons']    # slabs where ALL layers got ν
n_d11        = sc['d11']         # slabs that produced a D11 value

fail_density  = n_start    - n_density
fail_e_mod    = n_density  - n_e_mod
fail_poissons = n_e_mod    - n_poissons
fail_d11      = n_poissons - n_d11

def pct(n, d): return f"{n/d:.1%}" if d else "—"

# ── Node layout ───────────────────────────────────────────────────────────
#
# Column:   0      1      2      3      4
# x pos:  0.01   0.26   0.51   0.76   0.99
#
#  0  All slabs
#  1  Pass density   (high)    2  Fail density   (low, y=0.82)
#  3  Pass E-mod     (high)    4  Fail E-mod     (y=0.72, staggered)
#  5  Pass ν         (high)    6  Fail ν         (y=0.62, staggered)
#  7  Pass D11       (high)    8  Fail D11       (y=0.72)

BLUE  = "rgba( 68, 114, 196, 0.85)"   # density
GREEN = "rgba( 84, 168, 104, 0.85)"   # E-mod
PURP  = "rgba(148, 103, 189, 0.85)"   # Poisson's ratio
RED   = "rgba(196,  84,  78, 0.85)"   # D11
GREY  = "rgba(170, 170, 170, 0.65)"   # failures

node_labels = [
    f"All slabs<br>{n_start:,}",
    f"Pass density<br>{n_density:,} ({pct(n_density, n_start)})",
    f"Fail density<br>{fail_density:,} ({pct(fail_density, n_start)})",
    f"Pass E-mod<br>{n_e_mod:,} ({pct(n_e_mod, n_start)})",
    f"Fail E-mod<br>{fail_e_mod:,} ({pct(fail_e_mod, n_density)} of pass density)",
    f"Pass ν<br>{n_poissons:,} ({pct(n_poissons, n_start)})",
    f"Fail ν<br>{fail_poissons:,} ({pct(fail_poissons, n_e_mod)} of pass E-mod)",
    f"Pass D11<br>{n_d11:,} ({pct(n_d11, n_start)})",
    f"Fail D11<br>{fail_d11:,} ({pct(fail_d11, n_poissons)} of pass ν)",
]

#           0     1     2     3     4     5     6     7     8
node_x   = [0.01, 0.26, 0.26, 0.51, 0.51, 0.76, 0.76, 0.99, 0.99]
node_y   = [0.35, 0.18, 0.82, 0.18, 0.72, 0.18, 0.62, 0.18, 0.72]
node_col = [BLUE, BLUE, GREY, GREEN, GREY, PURP, GREY, RED,  GREY]

link_source = [0, 0, 1, 1, 3, 3, 5, 5]
link_target = [1, 2, 3, 4, 5, 6, 7, 8]
link_value  = [
    n_density,   fail_density,
    n_e_mod,     fail_e_mod,
    n_poissons,  fail_poissons,
    n_d11,       fail_d11,
]
link_color  = [
    "rgba( 68, 114, 196, 0.22)",
    "rgba(170, 170, 170, 0.20)",
    "rgba( 84, 168, 104, 0.22)",
    "rgba(170, 170, 170, 0.20)",
    "rgba(148, 103, 189, 0.22)",
    "rgba(170, 170, 170, 0.20)",
    "rgba(196,  84,  78, 0.22)",
    "rgba(170, 170, 170, 0.20)",
]

fig = go.Figure(go.Sankey(
    arrangement="snap",
    node=dict(
        label=node_labels,
        x=node_x, y=node_y,
        color=node_col,
        pad=12, thickness=20,
        line=dict(color="white", width=0.5),
    ),
    link=dict(
        source=link_source, target=link_target,
        value=link_value, color=link_color,
    ),
))

# ── Method label annotations ──────────────────────────────────────────────
# Labels sit below each destination column. ax/ay are pixel offsets from
# the annotation point (arrow tip is at the label, tail points upward).
# axref/ayref must be 'pixel' in this version of Plotly.

fig.update_layout(
    title=dict(
        text=(
            f"<b>Sequential calculation chain: {TARGET}</b><br>"
            f"<sup>Algorithm order: density → elastic modulus → Poisson’s ratio → D11  —  "
            f"total: {n_start:,} ECTP slabs</sup>"
        ),
        x=0.5, xanchor="center", font=dict(size=13),
    ),
    font=dict(size=11),
    width=1000,
    height=580,
    margin=dict(l=20, r=80, t=100, b=120),
    annotations=[
        # Labels placed below each output column using paper coords;
        # arrow drawn with pixel offset (ax=0 = directly above label).
        dict(
            x=0.26, y=-0.14, xref="paper", yref="paper",
            ax=0, ay=-25, axref="pixel", ayref="pixel",
            showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1.5,
            arrowcolor="rgba(68,114,196,0.7)",
            text=f"<b>{METHOD_DENSITY}</b><br><span style='font-size:10px'>density</span>",
            font=dict(size=11, color="rgba(68,114,196,1)"), align="center",
        ),
        dict(
            x=0.51, y=-0.14, xref="paper", yref="paper",
            ax=0, ay=-25, axref="pixel", ayref="pixel",
            showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1.5,
            arrowcolor="rgba(84,168,104,0.7)",
            text=f"<b>{METHOD_EMOD}</b><br><span style='font-size:10px'>elastic modulus</span>",
            font=dict(size=11, color="rgba(84,168,104,1)"), align="center",
        ),
        dict(
            x=0.76, y=-0.14, xref="paper", yref="paper",
            ax=0, ay=-25, axref="pixel", ayref="pixel",
            showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1.5,
            arrowcolor="rgba(148,103,189,0.7)",
            text=f"<b>{METHOD_POISSONS}</b><br><span style='font-size:10px'>Poisson’s ratio</span>",
            font=dict(size=11, color="rgba(148,103,189,1)"), align="center",
        ),
        dict(
            x=0.97, y=-0.14, xref="paper", yref="paper",
            ax=0, ay=-25, axref="pixel", ayref="pixel",
            showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=1.5,
            arrowcolor="rgba(196,84,78,0.7)",
            text=f"<b>{METHOD_D11}</b><br><span style='font-size:10px'>D11</span>",
            font=dict(size=11, color="rgba(196,84,78,1)"), align="center",
        ),
    ],
)

fig.show()
